# Стандартно подключаем датасеты AG-News и imdb

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_news0 = load_dataset("ag_news")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
count0, count1, count2, count3 = 0, 0, 0, 0
dataset_news = []
for i in range(len(dataset_news0['train'])):
  if dataset_news0['train'][i]['label'] == 0 and count0 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news0['train'][i]['label'] == 1 and count1 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news0['train'][i]['label'] == 2 and count2 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news0['train'][i]['label'] == 3 and count3 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_news)

8000

In [ ]:
import random
random.shuffle(dataset_news)

In [ ]:
# так как потом мне нужно будет использовать и кросс-валидацию, и разделеине на train/test, просто поделю на тексты и метки
dataset_news_texts = [i['news'] for i in dataset_news]
dataset_news_labels = [i['label'] for i in dataset_news]

In [ ]:
dataset_imdb0 = load_dataset("imdb")

count0, count1 = 0, 0
dataset_imdb = []

for i in range(len(dataset_imdb0['train'])):
  if dataset_imdb0['train'][i]['label'] == 0 and count0 < 4000:
    dataset_imdb.append({'text': dataset_imdb0['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb0['train'][i]['label'] == 1 and count1 < 4000:
    dataset_imdb.append({'text': dataset_imdb0['train'][i]['text'], 'label': 1})
    count1 += 1
len(dataset_imdb)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

8000

In [ ]:
random.shuffle(dataset_imdb)

In [ ]:
dataset_imdb_texts = [i['text'] for i in dataset_imdb]
dataset_imdb_labels = [i['label'] for i in dataset_imdb]

# Функции предобработки и стемминг/лемматизация

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords

nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = list(set(stopwords.words("english")))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preproccesing(text):
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if not word in stop_words]
  tokens = word_tokenize(re.sub(r'[^a-zA-Zа-яА-Я ]', '', ' '.join(tokens)))  # убираем спец символы, числа и знаки препинания
  return tokens

In [ ]:
# примеры базового preprocesing'а для обоих датасетов
print(dataset_news_texts[0])
print(preproccesing(dataset_news_texts[0]))
print()
print(dataset_imdb_texts[0])
print(preproccesing(dataset_imdb_texts[0]))

Wilkinson #39;s form cheers Woodward The optimism felt by Jonny Wilkinson following his successful comeback for Newcastle in Galway this week was echoed yesterday by a buoyant Sir Clive Woodward, who can now look forward to welcoming his prize asset back into the England fold next month. 
['wilkinson', 'form', 'cheers', 'woodward', 'optimism', 'felt', 'jonny', 'wilkinson', 'following', 'successful', 'comeback', 'newcastle', 'galway', 'week', 'echoed', 'yesterday', 'buoyant', 'sir', 'clive', 'woodward', 'look', 'forward', 'welcoming', 'prize', 'asset', 'back', 'england', 'fold', 'next', 'month']

Billed as Takashi Miike's "first family film" - by people who haven't seen Zebraman, presumably. YOKAI DAISENSO takes things even further in the direction of family-friendliness, diluting the darkness and cynicism to create a grand fantasy fairy tale. A young boy is chosen by fate to save the world from monsters and horrors of which they remain largely unaware. The film is evidently bigger budg

In [ ]:
# добавляем стемминг и лемматизацию
nltk.download('wordnet')
nltk.download('omw-1.4')

def stemming(text):
  tokens = preproccesing(text)  # базовый preproccesing
  stemmer = nltk.PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]  # перебираем токены и применяем алгоритм стемминга
  return stemmed_tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# примеры preproccesing'а после стемминга
print(dataset_news_texts[0])
print(stemming(dataset_news_texts[0]))
print()
print(dataset_imdb_texts[0])
print(stemming(dataset_imdb_texts[0]))

Wilkinson #39;s form cheers Woodward The optimism felt by Jonny Wilkinson following his successful comeback for Newcastle in Galway this week was echoed yesterday by a buoyant Sir Clive Woodward, who can now look forward to welcoming his prize asset back into the England fold next month. 
['wilkinson', 'form', 'cheer', 'woodward', 'optim', 'felt', 'jonni', 'wilkinson', 'follow', 'success', 'comeback', 'newcastl', 'galway', 'week', 'echo', 'yesterday', 'buoyant', 'sir', 'clive', 'woodward', 'look', 'forward', 'welcom', 'prize', 'asset', 'back', 'england', 'fold', 'next', 'month']

Billed as Takashi Miike's "first family film" - by people who haven't seen Zebraman, presumably. YOKAI DAISENSO takes things even further in the direction of family-friendliness, diluting the darkness and cynicism to create a grand fantasy fairy tale. A young boy is chosen by fate to save the world from monsters and horrors of which they remain largely unaware. The film is evidently bigger budget than anything

In [ ]:
def lemmatisation(text):
  tokens = preproccesing(text)
  lemma = nltk.WordNetLemmatizer()
  lemma_tokens = [lemma.lemmatize(token) for token in tokens]
  return lemma_tokens

In [ ]:
# примеры preproccesing'а после лемматизации
print(dataset_news_texts[0])
print(lemmatisation(dataset_news_texts[0]))
print()
print(dataset_imdb_texts[0])
print(lemmatisation(dataset_imdb_texts[0]))

Wilkinson #39;s form cheers Woodward The optimism felt by Jonny Wilkinson following his successful comeback for Newcastle in Galway this week was echoed yesterday by a buoyant Sir Clive Woodward, who can now look forward to welcoming his prize asset back into the England fold next month. 
['wilkinson', 'form', 'cheer', 'woodward', 'optimism', 'felt', 'jonny', 'wilkinson', 'following', 'successful', 'comeback', 'newcastle', 'galway', 'week', 'echoed', 'yesterday', 'buoyant', 'sir', 'clive', 'woodward', 'look', 'forward', 'welcoming', 'prize', 'asset', 'back', 'england', 'fold', 'next', 'month']

Billed as Takashi Miike's "first family film" - by people who haven't seen Zebraman, presumably. YOKAI DAISENSO takes things even further in the direction of family-friendliness, diluting the darkness and cynicism to create a grand fantasy fairy tale. A young boy is chosen by fate to save the world from monsters and horrors of which they remain largely unaware. The film is evidently bigger budge

# Каждому датасету создадим по 3 варианта: базовый preproccesing, стэмминг и лемматизация

In [ ]:
dataset_news_base = [' '.join(preproccesing(text)) for text in dataset_news_texts]
dataset_news_stem = [' '.join(stemming(text)) for text in dataset_news_texts]
dataset_news_lemma = [' '.join(lemmatisation(text)) for text in dataset_news_texts]

In [ ]:
dataset_imdb_base = [' '.join(preproccesing(text)) for text in dataset_imdb_texts]
dataset_imdb_stem = [' '.join(stemming(text)) for text in dataset_imdb_texts]
dataset_imdb_lemma = [' '.join(lemmatisation(text)) for text in dataset_imdb_texts]

# Создадим два vectorizers: униграммы и униграммы+биграммы

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect_uni = TfidfVectorizer(ngram_range=(1, 1))
vect_bi = TfidfVectorizer(ngram_range=(1, 2))

# Тестово обучаем RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

rf = RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

cv_strat_rf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 30],      # Максимальная глубина дерева
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 2, 4],        # Минимальное число образцов в листе
    'class_weight': [None, 'balanced']    # Балансировка классов
}

In [ ]:
grid_search_rf = GridSearchCV(
    rf,
    param_grid_rf,
    cv=cv_strat_rf,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=2
)

In [ ]:
all_datasets ={
    'news_base': (dataset_news_base, dataset_news_labels),
    'news_stem': (dataset_news_stem, dataset_news_labels),
    'news_lemma': (dataset_news_lemma, dataset_news_labels),
    'imdb_base': (dataset_imdb_base, dataset_imdb_labels),
    'imdb_stem': (dataset_imdb_stem, dataset_imdb_labels),
    'imdb_lemma': (dataset_imdb_lemma, dataset_imdb_labels)
}

In [ ]:
results_rf = []

# Перебор векторизаторов и датасетов
for vect_name, vect in [('Unigram', vect_uni), ('Bigram', vect_bi)]:
    for data_name, (data, labels) in all_datasets.items():
        print(f"Vectorizer: {vect_name}, dataset: {data_name}")

        # Векторизация (обучаем ТОЛЬКО на train данных в кросс-валидации)
        X = vect.fit_transform(data)

        # Поиск параметров
        grid_search_rf.fit(X, labels)

        # Лучшие параметры и метрики
        best_params = grid_search_rf.best_params_
        best_score = grid_search_rf.best_score_

        print(f"Best F1: {best_score:.4f}")
        print(f"Best params: {best_params}")
        print()

        # Сохраняем результаты
        results_rf.append({
            'Vectorizer': vect_name,
            'Dataset': data_name,
            'Best F1': best_score,
            'Best params': best_params
        })

# Результаты в DataFrame
results_df_rf = pd.DataFrame(results_rf)
print(results_df_rf)

Vectorizer: Unigram, dataset: news_base
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best F1: 0.8459
Best params: {'class_weight': None, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

Vectorizer: Unigram, dataset: news_stem
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best F1: 0.8523
Best params: {'class_weight': None, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Vectorizer: Unigram, dataset: news_lemma
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best F1: 0.8453
Best params: {'class_weight': None, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}

Vectorizer: Unigram, dataset: imdb_base
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best F1: 0.8545
Best params: {'class_weight': None, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}

Vectorizer: Unigram, data

# Обучаем LinearSVC
Сначала проведем обучение по стратифицированной кросс-валидации

In [ ]:
from sklearn.svm import LinearSVC
import pandas as pd
lin_scv = LinearSVC(C=1.0, random_state=42)  # параметры для SVM

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

cv_strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # параметры для кросс-валидации

# определяем сетку параметров
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [None, 'balanced'],
    'loss': ['hinge', 'squared_hinge'],
    'penalty': ['l2', 'l1'],
    'dual': [True, False],
    'multi_class': ['ovr', 'crammer_singer'],
    'max_iter': [1000, 2000]
}

# объединяем всё в GridSearchCV
grid_search = GridSearchCV(lin_scv, param_grid, cv=cv_strat, scoring='f1_weighted', verbose=1, n_jobs=-1)

In [ ]:
# перебор всех датасетов

all_datasets ={
    'news_base': (dataset_news_base, dataset_news_labels),
    'news_stem': (dataset_news_stem, dataset_news_labels),
    'news_lemma': (dataset_news_lemma, dataset_news_labels),
    'imdb_base': (dataset_imdb_base, dataset_imdb_labels),
    'imdb_stem': (dataset_imdb_stem, dataset_imdb_labels),
    'imdb_lemma': (dataset_imdb_lemma, dataset_imdb_labels)
}

In [ ]:
results = []

# Перебор векторизаторов и датасетов
for vect_name, vect in [('Unigram', vect_uni), ('Bigram', vect_bi)]:
    for data_name, (data, labels) in all_datasets.items():
        print(f"Vectorizer: {vect_name}, dataset: {data_name}")

        # Векторизация (обучаем ТОЛЬКО на train данных в кросс-валидации)
        X = vect.fit_transform(data)

        # Поиск параметров
        grid_search.fit(X, labels)

        # Лучшие параметры и метрики
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        print(f"Best F1: {best_score:.4f}")
        print(f"Best params: {best_params}")
        print()

        # Сохраняем результаты
        results.append({
            'Vectorizer': vect_name,
            'Dataset': data_name,
            'Best F1': best_score,
            'Best params': best_params
        })

# Результаты в DataFrame
results_df = pd.DataFrame(results)
print(results_df)

Vectorizer: Unigram, dataset: news_base
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
400 fits failed out of a total of 1600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/svm/_classes.py", line 321, in fit
    self.coef_, self.intercept_, 

Best F1: 0.8869
Best params: {'C': 1, 'class_weight': None, 'dual': True, 'loss': 'hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2'}

Vectorizer: Unigram, dataset: news_stem
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
400 fits failed out of a total of 1600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/svm/_classes.py", line 321, in fit
    self.coef_, self.intercept_, 

Best F1: 0.8860
Best params: {'C': 1, 'class_weight': None, 'dual': True, 'loss': 'hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2'}

Vectorizer: Unigram, dataset: news_lemma
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


...

Best F1: 0.8986
Best params: {'C': 100, 'class_weight': None, 'dual': False, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2'}

Vectorizer: Bigram, dataset: imdb_stem
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
400 fits failed out of a total of 1600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/svm/_classes.py", line 321, in fit
    self.coef_, self.intercept_, 

Best F1: 0.8966
Best params: {'C': 10, 'class_weight': None, 'dual': True, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2'}

Vectorizer: Bigram, dataset: imdb_lemma
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
400 fits failed out of a total of 1600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/svm/_classes.py", line 321, in fit
    self.coef_, self.intercept_, 

Best F1: 0.8977
Best params: {'C': 10, 'class_weight': None, 'dual': True, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2'}

   Vectorizer     Dataset   Best F1  \
0     Unigram   news_base  0.886856   
1     Unigram   news_stem  0.885961   
2     Unigram  news_lemma  0.886752   
3     Unigram   imdb_base  0.890112   
4     Unigram   imdb_stem  0.885366   
5     Unigram  imdb_lemma  0.887734   
6      Bigram   news_base  0.891209   
7      Bigram   news_stem  0.889448   
8      Bigram  news_lemma  0.890868   
9      Bigram   imdb_base  0.898607   
10     Bigram   imdb_stem  0.896608   
11     Bigram  imdb_lemma  0.897733   

                                          Best params  
0   {'C': 1, 'class_weight': None, 'dual': True, '...  
1   {'C': 1, 'class_weight': None, 'dual': True, '...  
2   {'C': 1, 'class_weight': None, 'dual': True, '...  
3   {'C': 1, 'class_weight': None, 'dual': True, '...  
4   {'C': 1, 'class_weight': None, 'dual': True, '... 